# Query the Dataset in S3 using Amazon Athena and AWS Glue Catalog
This will create an Athena table in the Glue Catalog (Hive Metastore) which allows us to query the dataset files in S3.  We will create a table in the Glue Catalog based on the `Amazon Customer Reviews Dataset` in S3.  We define the columns that map to the data, specify how the data is formatted, and provide the location in Amazon S3 for the file(s).


<img src="img/athena_register_parquet.png" width="60%" align="left">

In [ ]:
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

In [ ]:
from pyathena import connect
import pandas as pd

In [ ]:
%store -r setup_dependencies_passed

In [ ]:
try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
print(setup_dependencies_passed)

In [ ]:
if not setup_dependencies_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

# Create Athena Table for Parquet Data

#### Dataset columns

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.



In [ ]:
s3_staging_dir = "s3://{}/athena/query-cache".format(bucket)

In [ ]:
database_name = "default"
table_name = "amazon_reviews_parquet"

In [ ]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [ ]:
# SQL statement to execute
statement = """
    CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
      marketplace string, 
      customer_id string, 
      review_id string, 
      product_id string, 
      product_parent string, 
      product_title string, 
      star_rating int, 
      helpful_votes int, 
      total_votes int, 
      vine string, 
      verified_purchase string, 
      review_headline string, 
      review_body string, 
      review_date bigint, 
      year int)
    PARTITIONED BY (product_category string)
    ROW FORMAT SERDE 
      'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe' 
    STORED AS INPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetInputFormat' 
    OUTPUTFORMAT 
      'org.apache.hadoop.hive.ql.io.parquet.MapredParquetOutputFormat'
    LOCATION
      's3://amazon-reviews-pds/parquet/'
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)

# Analyze and Build the Partitions

In [ ]:
statement = """
    MSCK REPAIR TABLE {}.{}
""".format(
    database_name, table_name
)

print(statement)

pd.read_sql(statement, conn)

# Verify The Table Has Been Created Succesfully

In [ ]:
statement = "SHOW TABLES in {}".format(database_name)

df_tables = pd.read_sql(statement, conn)
df_tables

# Run A Sample Query

In [ ]:
product_category = "Digital_Software"

statement = """
    SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100
""".format(
    database_name, table_name, product_category
)

print(statement)

In [ ]:
df = pd.read_sql(statement, conn)
df.head(5)

In [ ]:
if df.empty:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOUR DATA HAS NOT BEEN REGISTERED WITH ATHENA.")
    print("LOOK IN PREVIOUS CELLS TO FIND THE ISSUE.             ")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

In [ ]:
if "amazon_reviews_parquet" in df_tables.values and not df.empty:
    ingest_create_athena_table_parquet_passed = True

In [ ]:
%store ingest_create_athena_table_parquet_passed

# Review the New Athena Table in the Glue Catalog

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="top" href="https://console.aws.amazon.com/glue/home?region={}#table:catalog={};name=amazon_reviews_parquet;namespace=default">AWS Glue Catalog</a></b>'.format(
            region, account_id
        )
    )
)

# Store Variables for the Next Notebooks

In [ ]:
%store

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>